# MODFLOW 6: inicializando una simulación.


## Resumen.

El propósito de este documento es describir cómo se realiza una simulación de flujo usando el modelo GWF de MODFLOW 6. Esta descripción combina los conceptos descritos en la documentación de MODFLOW 6 y en el software [***flopy***](https://www.usgs.gov/software/flopy-python-package-creating-running-and-post-processing-modflow-based-models) ([repositorio](https://github.com/modflowpy/flopy), [documentación](https://flopy.readthedocs.io/en/3.3.2/index.html)). El software flopy permite simplificar la generación de los archivos de entrada para una simulación a través de Python, además de realizar la ejecución de la simulación y el post-procesamiento de la salida. No se hace una descripción detallada, sino que solo se explican los conceptos principales y se relacionan con los archivos de entrada requeridos por MODFLOW 6 y con los objetos de flopy. Se hace una configuración de una simulación agregando las componentes *Timing module*, *Numerical Solution* (IMS) y un *GWF model*, usando parámetros reducidos.

<p xmlns:cc="http://creativecommons.org/ns#" xmlns:dct="http://purl.org/dc/terms/"><a property="dct:title" rel="cc:attributionURL" href="https://github.com/luiggix/mf6_tutorial/">MODFLOW 6: tutorial</a> (0_mf6/01_GWF_init.ipynb) by <b>Luis M. de la Cruz Salas (2025)</b> is licensed under <a href="http://creativecommons.org/licenses/by-sa/4.0/?ref=chooser-v1" target="_blank" rel="license noopener noreferrer" style="display:inline-block;">Attribution-ShareAlike 4.0 International<img style="height:22px!important;margin-left:3px;vertical-align:text-bottom;" src="https://mirrors.creativecommons.org/presskit/icons/cc.svg?ref=chooser-v1"><img style="height:22px!important;margin-left:3px;vertical-align:text-bottom;" src="https://mirrors.creativecommons.org/presskit/icons/by.svg?ref=chooser-v1"><img style="height:22px!important;margin-left:3px;vertical-align:text-bottom;" src="https://mirrors.creativecommons.org/presskit/icons/sa.svg?ref=chooser-v1"></a>.</p> 

<div class="alert alert-block alert-info">

## Ejemplo. Las tres componentes de una simulación con flopy.

Una simulación simple con GWF requiere de las componentes que se muestran en la siguiente figura:

<figure>
  <img src="../figures/modflow00.png" width=300px hspace="5" vspace="5"/>
  <figcaption>Figura 1. Diagrama que muestra las componentes que se usan para la simulación de un solo modelo GWF. Las componentes son: (1) una simulación (que es el programa principal), (2) una componente para gestionar el tiempo, (3) una componente para gestionar la solución numérica y (4) un único modelo GWF. Figura tomada de [1].
  </figcaption>
</figure> 

Para definir las tres componentes mostradas en la figura anterior haremos uso de la biblioteca *flopy*. Realizaremos este proceso para un ejemplo simple, cuya única utilidad es explicar el proceso (sin resolver un problema de flujo).

</div>

### Paso 1. Inicialización de la simulación.

La simulación se inicia creando un objeto de la clase [`flopy.mf6.MFSimulation`](https://flopy.readthedocs.io/en/3.3.2/source/flopy.mf6.modflow.mfsimulation.html) para cargar, construir y salvar los archivos de una simulación de MODFLOW 6. Se debe crear un objeto de este tipo antes que cualquier otro objeto de las componentes o paquetes que se vayan a usar en la simulación.

A continuación creamos el objeto `o_sim` de la clase `flopy.mf6.MFSimulation` como sigue:

In [1]:
# Solo requerimos incluir la biblioteca flopy
import flopy

In [2]:
# --- Ruta a los ejecutables de MODFLOW 6. Opciones de SO's: linux, macos, macosarm, windows
exe_name = "C:\\Users\\luiggi\\Documents\\GitSites\\mf6_tutorial\\mf6\\windows\\mf6"
#exe_name = "../../mf6/macosarm/mf6"

# Creación del objeto 'o_sim' para la simulación; es el que controla todo.
o_sim = flopy.mf6.MFSimulation(
    sim_name = "flow", 
    exe_name = exe_name, 
    sim_ws   = "sandbox1" 
)

* Al ejecutar la celda de código anterior, se crea la carpeta `sandbox1` en donde se almacenarán todos los archivos de entrada y de salida que genera MODFLOW 6. Este es el espacio de trabajo (*workspace*) que por ahora estará vacío.

* El nombre de la simulación será `flow` y este nombre será usado para generar archivos de entrada y salida con las extensiones correspondientes.

* Una vez creado un objeto de esta clase, los demás objetos que contribuyen a la simulación se deben enlazar al objeto de la simulación `o_sim`.

* Con la instrucción `print(o_sim)` es posible imprimir información de los atributos del objeto `o_sim`, como se hace en la siguiente celda:


In [3]:
print(o_sim)

sim_name = flow
sim_path = C:\Users\luiggi\Documents\GitSites\mf6_tutorial\examples\0_mf6\sandbox1
exe_name = C:\Users\luiggi\Documents\GitSites\mf6_tutorial\mf6\windows\mf6

###################
Package mfsim.nam
###################

package_name = mfsim.nam
filename = mfsim.nam
package_type = nam
model_or_simulation_package = simulation
simulation_name = flow





* También, con el método `write_simulation()` es posible escribir en archivos la información que usará MODFLOW 6 como entrada para realizar la simulación (no te preocupes si obtienes un error, esto se explica más adelante):

In [4]:
o_sim.write_simulation()

writing simulation...
  writing simulation name file...
  writing simulation tdis package...


AttributeError: 'NoneType' object has no attribute 'write'

* Observa que se obtiene un error (`AttributeError`) y esto es debido a que se requiere de agregar la componente para la gestión del tiempo de la simulación. Esto lo haremos en el siguiente paso.
* Se genera el archivo `mfsim.nam`, el cual es necesario para ejecutar la simulación. Este archivo se almacena en la carpeta del espacio de trabajo, en este caso en `sandbox1` y  por ahora contiene muy poca información; algo similar a lo siguiente:
```
# File generated by Flopy version 3.9.2 on 05/09/2025 at 14:31:30.
BEGIN options
END options

BEGIN exchanges
END exchanges
```

* Este archivo se actualizará conforme se agreguen componentes a la simulación.

### Paso 2. Discretización temporal.

Es necesario definir los parámetros para gestionar el tiempo de la simulación. En este caso usaremos:

* Unidades: `DAYS`
* Periodos de estrés: `NPER` $= 1$
* Datos del periodo de estrés: `(PERLEN, NSTP, TSMULT )` $ = (1.0, 1, 1.0)$

Ejecutamos la siguiente celda:

In [6]:
o_tdis = flopy.mf6.ModflowTdis(
    simulation = o_sim,
    time_units = "DAYS",
    nper = 1,
    perioddata = [(1.0, 1, 1.0)]
)


* En la celda anterior se construye el objeto `o_tdis` que es de tipo [`flopy.mf6.ModflowTdis`](https://flopy.readthedocs.io/en/latest/source/flopy.mf6.modflow.mftdis.html).
* El primer parámetro es el objeto `o_sim`, es decir la simulación. De esta manera la simulación conoce los parámetros de la discretización del tiempo.
* El parámetro `perioddata` es una lista que contiene tuplas, cada una de ellas con los datos `(PERLEN, NSTP, TSMULT)` para cada periodo de estrés.
* También es posible usar la instrucción `print()` para imprimir información de los atributos de este objeto, veamos:
</div>

In [7]:
print(o_tdis)

package_name = flow.tdis
filename = flow.tdis
package_type = tdis
model_or_simulation_package = simulation
simulation_name = flow

Block options
--------------------
time_units
{internal}
(days)


Block dimensions
--------------------
nper
{internal}
(1)


Block perioddata
--------------------
perioddata
{internal}
([(1., 1, 1.)])





* Obsérva que se usa el nombre de archivo `filename = flow.tdis` para almacenar la información del tiempo, esto es porque el nombre de la simulación es `flow`.
* Para generar este archivo hacemos lo siguiente:

In [8]:
o_sim.write_simulation()

writing simulation...
  writing simulation name file...
  writing simulation tdis package...


* La instrucción anterior actualiza el archivo `mfsim.nam`. Este archivo contendrá información de los objetos (componentes) que contribuyen a la simulación. Después de ejecutar la celda anterior el contenido de este archivo debe ser similar a lo siguiente:

```
# File generated by Flopy version 3.9.2 on 04/20/2025 at 12:02:33.
BEGIN options
END options

BEGIN timing
  TDIS6  flow.tdis
END timing

BEGIN exchanges
END exchanges
```
* Nota que solo se ha incluido la información del módulo de tiempo. Esta información será actualizada más adelante.
* Se genera también el archivo `flow.tdis` con la información para la discretización temporal. El contenido de este archivo es como sigue:

```
# File generated by Flopy version 3.9.2 on 04/20/2025 at 12:02:33.
BEGIN options
  TIME_UNITS  days
END options

BEGIN dimensions
  NPER  1
END dimensions

BEGIN perioddata
       1.00000000  1       1.00000000
END perioddata
```

### Paso 3. Solución numérica.

Para cada modelo se requiere un objeto que calcule la solución numérica. 

En la celda que sigue se hace lo siguiente: 
* se define un objeto de la clase [`flopy.mf6.ModflowIms`](https://flopy.readthedocs.io/en/3.3.2/source/flopy.mf6.modflow.mfims.html),
* se imprime la información con la función `print()`y
* se escriben los archivos correspondientes.


In [9]:
o_ims = flopy.mf6.ModflowIms(simulation = o_sim)

print(o_ims)

o_sim.write_simulation()

package_name = ims_-1
filename = flow.ims
package_type = ims
model_or_simulation_package = simulation
simulation_name = flow


writing simulation...
  writing simulation name file...
  writing simulation tdis package...
  writing solution package ims_-1...


* El objeto `o_ims` representa a la solución numérica; el único parámetro que se usó en este ejemplo fue el objeto de la simulación para ligar la solución numérica con el modelo que se va a resolver.
* Se pueden agregar muchos más parámetros para configurar los métodos iterativos de solución.
* En este caso la información se guarda en el archivo `flow.ims` y su contenido es mínimo debido a que no usan más parámetros, por ejemplo:

```
# File generated by Flopy version 3.9.2 on 04/20/2025 at 12:18:32.
BEGIN options
END options
```
</div>

### Paso 4. Modelo GWF.

Ahora vamos a agregar un modelo numérico a la simulación. Para ello creamos un objeto de la clase [`flopy.mf6.ModflowGwf`](https://flopy.readthedocs.io/en/3.3.2/source/flopy.mf6.modflow.mfgwf.html), imprimimos sus parámetros en pantalla y generamos los archivos correspondientes como sigue:

In [10]:
o_gwf = flopy.mf6.ModflowGwf(
    simulation = o_sim,
    modelname = "flow",
    model_nam_file = "flow.nam",
)

print(o_gwf)

o_sim.write_simulation()

name = flow
model_type = gwf6
version = mf6
model_relative_path = .


writing simulation...
  writing simulation name file...
  writing simulation tdis package...
  writing solution package ims_-1...
  writing model flow...
    writing model name file...


* Observa que el primer parámetro también es el objeto de la simulación `o_sim`.
* Adicionalmente se agrega el nombre del modelo, que en este caso es igual al de la simulación, y el nombre del archivo donde se guardará la información del modelo GWF.
* El código de la celda anterior crea el archivo `flow.nam` que por ahora tiene información reducida como la que sigue:

```
# File generated by Flopy version 3.9.2 on 04/20/2025 at 12:34:26.
BEGIN options
END options
```

* Más adelante, este archivo se actualizará con la información de los paquetes que se requieran para realizar la simulación. 

* La instrucción `o_sim.write_simulation()` actualiza también el archivo `mfsim.nam` con la información del solucionador y del modelo GWF, se debe tener algo similar a lo siguiente:

```
# File generated by Flopy version 3.9.2 on 04/20/2025 at 12:25:36.
BEGIN options
END options

BEGIN timing
  TDIS6  flow.tdis
END timing

BEGIN models
  gwf6  flow.nam  flow
END models

BEGIN exchanges
END exchanges

BEGIN solutiongroup  1
  ims6  flow.ims  flow
END solutiongroup  1
```

* Observa que ahora este archivo contiene la información de las tres componentes necesarias para iniciar una simulación: Timing, Models y SolutionGroup. En este ejemplo no se agregan intercambios, pues se trata de un solo modelo.

Más información acerca de los archivos de entrada y salida de MODFLOW 6 se puede encontrar en [2].

En este ejemplo se ha construido el esquema requerido para iniciar una simulación de flujo con GWF de MODFLOW 6 usando las herramientas de *flopy*. Para que la simulación tenga más sentido, se requiere de agregar paquetes al modelo GWF, lo cual se hará en la notebook [02_GWF_paq.ipynb](02_GWF_paq.ipynb).

# Referencias

[1] Langevin, C. D., Hughes, J. D., Provost, A. M., Russcher, M. J., & Panday, S. (2023). MODFLOW as a configurable Multi‐Model Hydrologic Simulator. Ground Water. https://doi.org/10.1111/gwat.13351.

[2] MODFLOW 6 – Description of Input and Output. Version mf6.4.4—February 13, 2024. U.S. Department of the Interior. U.S. Geological Survey.. Archivo: mf6io.pdf de la documentación de MODFLOW 6 que se puede obtener de https://github.com/MODFLOW-ORG/modflow6/releases.